In [190]:
import requests, json
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
import asyncio
import os
import json
import copy

from anchorpy import Wallet
from anchorpy import Provider
from solana.keypair import Keypair
from solana.rpc.async_api import AsyncClient

from driftpy.constants.config import configs
from driftpy.types import *
#MarketType, OrderType, OrderParams, PositionDirection, OrderTriggerCondition

from driftpy.clearing_house import ClearingHouse
from driftpy.constants.numeric_constants import BASE_PRECISION, PRICE_PRECISION

from driftpy.constants.config import configs
from anchorpy import Provider
from anchorpy import Wallet
from solana.rpc.async_api import AsyncClient
from driftpy.clearing_house import ClearingHouse
from driftpy.clearing_house_user import ClearingHouseUser
from driftpy.accounts import *
from solana.keypair import Keypair
from driftpy.math.positions import is_available
from driftpy.constants.numeric_constants import *
import os



MARKET_INDEX_SOL = 0
userAccount = "ECbtn9Y34m6L89EBDMRYxhpyrFgSHEcbELgozpqAvcDY"
PKEY = "7f6d3DWGkNrKTERtW9McbBQCVbZibuHUf7CsdUjJyK1t"
URL_ORDERS_SELF   = f"https://master.api.drift.trade/orders/userAccounts?userPublicKeys={PKEY}&pageIndex=0&pageSize=100&marketType=perp"
## (below), putting public key only allows me to see
URL_HISTORY_SELF  = f"https://beta.drift.trade/overview/history/tradeHistory?userAccount={PKEY}"
URL_DEVNET_OB     = "https://master.dlob.drift.trade/orders/json"

In [191]:
from driftpy import constants
constants.markets.devnet_markets

[Market(symbol='SOL-PERP', base_asset_symbol='SOL', market_index=0, pyth_oracle=J83w4HKfqxwcq3BEMMkPFSppX3gqekLyLJBexebFVkix),
 Market(symbol='BTC-PERP', base_asset_symbol='BTC', market_index=1, pyth_oracle=HovQMDrbAgAYPCmHVSrezcSmkMtXSSUsLDFANExrZh2J),
 Market(symbol='ETH-PERP', base_asset_symbol='ETH', market_index=2, pyth_oracle=EdVCmQ9FSPcVe5YySXDPCRmc8aDQLKJ9xvYBMZPie1Vw)]

In [225]:
def get_orderbook(perp_market="SOL-PERP"):
    """ Queries DEVNET orderbok for specified perp market, organizes data into bid/asks sorted
    """
    try:
        r = requests.get(URL_DEVNET_OB).json()
    except:
        print(r)
        raise Exception("Unable to fetch orderbook")

    slot    = r['slot']
    oracles = r['oracles']
    orders  = r['orders']

    ## market index -> price
    market_to_oracle_map = pd.DataFrame(r['oracles']).set_index('marketIndex').to_dict()['price']

    ## Get list of outstanding orders    
    orders_all = list(map(lambda d: {"user":d['user'][:10], **d.pop('order')}, orders))
    df = pd.DataFrame(orders_all)
    df_raw = pd.DataFrame(orders_all)
    
    ## Only interested in (a) SOL-PERP (b) limit orders
    # SOL is marketINdex=0
    df = df[(df.marketIndex == MARKET_INDEX_SOL) & (df.marketType=='perp')]
    
    df["oraclePrice"] = df["marketIndex"].apply(lambda x: market_to_oracle_map.get(x, None))

    ## Convert from lamports:
    for col in ['price', 'oraclePrice', 'oraclePriceOffset']:
        df[col] = df[col].astype(int)
        df[col] /= 1e6

    for col in ['quoteAssetAmountFilled']:
        df[col] = df[col].astype(int)
        df[col] /= 1e6 

    for col in ['baseAssetAmount', 'baseAssetAmountFilled']:
        df[col] = df[col].astype(int)
        df[col] /= 1e9
        
    
    bid_df = df[(df.direction == 'short') & (df.orderType == 'limit')]
    ask_df = df[(df.direction == 'long')   &( df.orderType == 'limit')]
    
    ## Fixed prices are ok
    ## Floating prices need to be marked to a fixed price according to oracle price    
    bid_float = bid_df.loc[bid_df.price == 0] # inv: 0 price <-> floating order .w.r.t oracle 
    ask_float = ask_df.loc[ask_df.price == 0] #
    
    bid_df.loc[bid_df.price == 0, "price"] = bid_float["oraclePrice"] + bid_float["oraclePriceOffset"]
    ask_df.loc[ask_df.price == 0, "price"] = ask_float["oraclePrice"] + ask_float["oraclePriceOffset"]
    
    
    bid_df = bid_df.sort_values(['price'], ascending=False)
    ask_df = ask_df.sort_values(['price'])
    
    return df_raw, bid_df, ask_df

def orderbook_simple(perp_market="SOL-PERP"):
    """
        orderbook with bid / ask at each price point, sorted in proper order.
        does not contain ids/ trigger levels  / etc
        meant to mimic the UI
    """
    _, bid, ask = get_orderbook(perp_market)
    
    
    
    bid_simple = bid[["price", "baseAssetAmount", "baseAssetAmountFilled", "postOnly", "oraclePriceOffset", "oraclePrice"]]
    ask_simple = ask[["price", "baseAssetAmount", "baseAssetAmountFilled", "postOnly", "oraclePriceOffset", "oraclePrice"]]

    #### SOME ARE PARTIALLY FILLED
    bid_simple["qty"] = bid_simple

    return bid_simple, ask_simple

df_raw, bid, ask = get_orderbook()
print(f"#bids={len(bid)}, #asks={len(ask)}")
bid_s, ask_s = orderbook_simple()

#bids=2, #asks=14


In [226]:
ask_s

,price,baseAssetAmount,baseAssetAmountFilled,postOnly,oraclePriceOffset,oraclePrice
16,0.100000,10.00,0.00,False,0.000000,23.241697
17,0.100000,1.00,0.00,False,0.000000,23.241697
12,10.000000,5.00,0.00,False,0.000000,23.241697
13,15.000000,10.00,0.00,False,0.000000,23.241697
8,18.000000,20.00,0.00,False,0.000000,23.241697
2,19.000000,2.33,0.00,False,0.000000,23.241697
5,19.000000,2.33,0.00,False,0.000000,23.241697
15,20.000000,12.22,0.00,False,0.000000,23.241697
88,20.000000,0.96,0.00,True,0.000000,23.241697
83,21.400000,1.00,0.00,False,0.000000,23.241697


In [3]:
async def get_solana_perp_oracle():
    """ return SOL-PERP oracle """
    with open(os.path.expanduser("/home/sjl/.config/solana/key.json"), 'r') as f: secret = json.load(f)
    kp = Keypair.from_secret_key(bytes(secret))
    subaccount = 0
    print('using public key:', kp.public_key, " subaccount = ", subaccount)

    env = 'devnet'
    config = configs[env]
    wallet = Wallet(kp) 
    connection = AsyncClient(config.default_http)
    provider = Provider(connection, wallet)

    ch = ClearingHouse.from_config(config, provider)
    chu = ClearingHouseUser(ch, authority=ch, subaccount_id=subaccount)
    perp_market = await chu.get_perp_market(0)
    oracle = (await chu.get_perp_oracle_data(perp_market)).price / PRICE_PRECISION

    return oracle

#import time
#while(True):
    #r = await get_solana_perp_oracle()
    #print(r)
    #time.sleep(10)

In [ ]:
epoch = 0

while True:
    print("Epoch: ", epoch)
    epoch += 1
    
    # Step 1: Get the current order book
    #order_book = await get_order_book(client, symbol)

    # Step 2: Calculate the fair price based on the order book
    #fair_price = calculate_fair_price(order_book)

    # Step 3: Calculate the bid and ask prices based on the fair price and desired spread
    #bid_price, ask_price = calculate_bid_ask(fair_price, spread)

    # Step 4: Cancel existing orders (if any)
    #await cancel_orders(client, symbol)

    # Step 5: Place new orders at the calculated bid and ask prices
    # If there are existing orders, this step will replace them with new orders
    #await place_orders(client, symbol, bid_price, ask_price, order_size)

    # Steps 4 and 5 combined:
    await cancel_and_place_orders(client, symbol, bid_price, ask_price, order_size)

    # Step 6: Wait for 1 second before repeating the loop
    await asyncio.sleep(1)
